In [3]:
import pandas
from tabulate import tabulate

In [4]:
structure = pandas.DataFrame(columns=['name', 'description',
                                      'causality', 'start',
                                      'type', 'unit'])
# Open P, Q, V for each PV
node = 7
structure = structure.append(
    {'name': 'KW_' + str(node),
     'causality': 'input'},
    ignore_index=True)
structure = structure.append(
    {'name': 'KVAR_' + str(node),
     'causality': 'input'},
    ignore_index=True)
structure = structure.append(
    {'name': 'Vpu_' + str(node),
     'causality': 'output'},
    ignore_index=True)
structure = structure.append(
    {'name': 'Vpu_' + str(12),
     'causality': 'output'},
    ignore_index=True)


structure['start'] = [0] * len(structure)
structure['description'] = ['_'] * len(structure)
structure['type'] = ['Real'] * len(structure)
structure['unit'] = ['_'] * len(structure)

print(tabulate(structure.head(), headers='keys', tablefmt='psql'))
structure.to_excel('pandapower/pandapower.xlsx', index=False)

+----+--------+---------------+-------------+---------+--------+--------+
|    | name   | description   | causality   |   start | type   | unit   |
|----+--------+---------------+-------------+---------+--------+--------|
|  0 | KW_7   | _             | input       |       0 | Real   | _      |
|  1 | KVAR_7 | _             | input       |       0 | Real   | _      |
|  2 | Vpu_7  | _             | output      |       0 | Real   | _      |
|  3 | Vpu_12 | _             | output      |       0 | Real   | _      |
+----+--------+---------------+-------------+---------+--------+--------+


In [9]:
import shlex, subprocess
cmd = ("C:/JModelica.org-2.1/setenv.bat && " +
       " cd pandapower/" + " && "
       "cyderc " +
       " --path ./"
       " --name pandapower" +
       " --io pandapower.xlsx" +
       " --fmu_struc python" +
       " --fmu_type cs" +
       " --path_to_simulatortofmu C:/Users/cyder/Desktop/" +
       "SimulatorToFMU/simulatortofmu/parser/SimulatorToFMU.py")        
args = shlex.split(cmd)
process = subprocess.Popen(args, bufsize=1, universal_newlines=True)
process.wait()
process.kill()

In [1]:
# Load both pandapower and pv fmus
from pyfmi import load_fmu
pandapower = load_fmu('pandapower/pandapower.fmu', kind='cs')
pv = load_fmu('pv_inverter/Pv_Inv_VoltVarWatt_simple_Slim.fmu', kind='cs')

In [2]:
print('inputs')
pandapower_inputs = pandapower.get_model_variables(causality=2).keys()
print(pandapower_inputs)
print('outputs')
pandapower_outputs = pandapower.get_model_variables(causality=3).keys()
print(pandapower_outputs)

inputs
['KVAR_7', 'KW_7']
outputs
['Vpu_12', 'Vpu_7']


In [3]:
print('inputs')
pv_inputs = pv.get_model_variables(causality=2).keys()
print(pv_inputs)
print('outputs')
pv_outputs = pv.get_model_variables(causality=3).keys()
print(pv_outputs)

inputs
['v']
outputs
['Q', 'P']


In [4]:
pandapower_input_ref = [pandapower.get_variable_valueref(
    pandapower_inputs[i]) for i in range(0, len(pandapower_inputs))]
pandapower_output_ref = [pandapower.get_variable_valueref(
    pandapower_outputs[i]) for i in range(0, len(pandapower_outputs))]
pv_input_ref = [pv.get_variable_valueref(
    pv_inputs[i]) for i in range(0, len(pv_inputs))]
pv_output_ref = [pv.get_variable_valueref(
    pv_outputs[i]) for i in range(0, len(pv_outputs))]
pandapower_input_ref

[56, 55]

In [5]:
pandapower.initialize()

0.0,2019-03-19 15:45:28.904000,0.0,0.0,0.9945337245296682,0.9751283601661919



No handlers could be found for logger "pandapower.auxiliary"


In [6]:
pv_inverter_parameters = {
    'weather_file':("C:\\Users\\cyder\\Desktop\\Jonathan\\" +
        'demo_cs_getting_started\\pv_inverter\\' +
        'USA_CA_San.Francisco.Intl.AP.724940_TMY3.mos'),
    'n': 1,
    'A': (2000 * 1000) / (0.158 * 1000),
    'eta': 0.158,
    'lat': 37.9,
    'til': 10,
    'azi': 0,
    'thrP': 0.05,  #0.05,
    'hysP': 0.04,  #0.04,
    'thrQ': 0.04,  #0.04,
    'hysQ': 0.01,  #0.01,
    'SMax': 2000*1.05,
    'QMaxInd': 2000*1.05*0.44,
    'QMaxCap': 2000*1.05*0.44,
}

for key, value in pv_inverter_parameters.items():
    pv.set(key, value)

In [7]:
pv.setup_experiment(start_time=0, stop_time=360*24*3600)
pv.initialize()

In [9]:
pv.set_real(pv_input_ref, [1.04])
pv.do_step(current_t=0, step_size=60, new_step=0)
tmp = list(pv.get_real(pv_output_ref))
print(tmp)

[-924.0005537597334, 0.00019450916252126456]


In [11]:
pandapower.set_real(pandapower_input_ref, [-924.0005537597334, 0.00019450916252126456])
pandapower.do_step(current_t=60, step_size=2, new_step=1)
tmp2 = list(pandapower.get_real(pandapower_output_ref))
print(tmp2)

62.0,2019-03-19 15:49:17.767000,0.000194509162521,-924.00055376,0.9806425602129208,0.9609495236706906

62.0,2019-03-19 15:49:17.827000,0.000194509162521,-924.00055376,0.9806425602129208,0.9609495236706906

[0.960949541016578, 0.9806425779414617]
